In [13]:
# Load Packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

In [14]:
# OS setup
main_path = os.getcwd()
data_path = os.path.join(main_path, 'data')
csv_path = os.path.join(main_path, 'info.csv') # csv 파일을 저장할 위치

if not os.path.exists(data_path):
    os.mkdir(data_path)

In [15]:
# Save empty csv
if not os.path.exists(csv_path):
    df = pd.DataFrame(columns=['state', 'gender', 'age', 'source', 'file'])
    df.to_csv(csv_path)

In [16]:
%run /Users/jaewone/developer/tensorflow/baby-cry-classification/donateacry_corpus_data_trans.ipynb

belly_pain: 16
discomfort: 27
burping: 8
tired: 24
hungry: 382

Count total: 457
Column: gender
Value "m" with counts 276
Value "f" with counts 181

Column: age
Value "26" with counts 162
Value "04" with counts 149
Value "72" with counts 81
Value "48" with counts 55
Value "22" with counts 10

Column: state
Value "bp" with counts 16
Value "dc" with counts 20
Value "ch" with counts 7
Value "bu" with counts 8
Value "ti" with counts 24
Value "hu" with counts 382



/var/folders/10/9_p98m6j42n84y4wmf_k0krc0000gn/T/ipykernel_60910/212709108.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  origin_df['file'] = origin_df['file'].str.replace(origin_file_path, '').str.replace(
